# 03_text_processing_and_split.ipynb – Textverarbeitung & Datensatz-Splitting

**Ziel:** Die preprocessed Amazon Reviews werden mit spaCy tokenisiert, von Stopwords bereinigt, lemmatisiert und anschließend stratifiziert in Trainings-, Validierungs- und Test­datensätze (80/10/10) aufgeteilt. Die Splits werden als .jsonl-Dateien im Ordner `data/processed/` gespeichert.

---


## 1. Setup und Laden der preprocessed Daten
Wir laden die bereinigten Datensätze aus `data/preprocessed`.


In [1]:
import pandas as pd
import json
from pathlib import Path

PRE_DIR = Path('../data/preprocessed')
CATEGORIES = ['Automotive', 'Video_Games', 'Books']
dfs = {}
for cat in CATEGORIES:
    path = PRE_DIR / f'{cat}_preprocessed.jsonl'
    df = pd.read_json(path, lines=True)
    dfs[cat] = df
    print(f'{cat}: {df.shape[0]} Einträge geladen.')


Automotive: 996426 Einträge geladen.
Video_Games: 995507 Einträge geladen.
Books: 997498 Einträge geladen.


## 2. Tokenisierung, Stopword-Entfernung & Lemmatisierung mit spaCy
Wir nutzen spaCy, um die Texte zu verarbeiten.



In [2]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

def spacy_preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

for cat, df in dfs.items():
    print(f'Bearbeite {cat} ...')
    df['text_processed'] = df['text'].apply(spacy_preprocess)
    dfs[cat] = df
    print(f'{cat}: Verarbeitung abgeschlossen.')


Bearbeite Automotive ...
Automotive: Verarbeitung abgeschlossen.
Bearbeite Video_Games ...
Video_Games: Verarbeitung abgeschlossen.
Bearbeite Books ...
Books: Verarbeitung abgeschlossen.


## 3. Stratifiziertes Splitten in Train/Valid/Test
Wir teilen die Daten im Verhältnis 80/10/10 auf und achten auf die Verteilung der Ratings. Die Splits werden im Ordner `data/processed/` gespeichert.


In [3]:
from sklearn.model_selection import train_test_split
SPLIT_DIR = Path('../data/processed')
SPLIT_DIR.mkdir(parents=True, exist_ok=True)

for cat, df in dfs.items():
    # 80 % Train, 20 % Temp
    train_df, temp_df = train_test_split(
        df, test_size=0.2, stratify=df['rating'], random_state=42
    )
    # Temp halbieren → 10 % Valid, 10 % Test
    valid_df, test_df = train_test_split(
        temp_df, test_size=0.5, stratify=temp_df['rating'], random_state=42
    )
    # Speichern
    train_df.to_json(SPLIT_DIR / f'{cat}_train.jsonl', orient='records',
                     lines=True, force_ascii=False)
    valid_df.to_json(SPLIT_DIR / f'{cat}_valid.jsonl', orient='records',
                     lines=True, force_ascii=False)
    test_df.to_json(SPLIT_DIR / f'{cat}_test.jsonl', orient='records',
                    lines=True, force_ascii=False)
    print(f'{cat}: Splits gespeichert unter {SPLIT_DIR}')


Automotive: Splits gespeichert unter ..\data\processed
Video_Games: Splits gespeichert unter ..\data\processed
Books: Splits gespeichert unter ..\data\processed
